In [2]:
!pip install tensorflow nltk


Data Collection and Preprocessing

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Sample data (extend this with your own dataset)
english_texts = ["Hello world", "How are you?", "Good morning", "Good night", "Thank you"]
amharic_texts = ["ሰላም ልዑል", "እንዴት ነህ?", "እንኳን ደህና አደረህ", "መልካም ሌሊት", "አመሰግናለሁ"]

# Function to preprocess texts
def preprocess_texts(texts):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded_sequences = pad_sequences(sequences, padding='post')
    return padded_sequences, tokenizer

# Preprocess English and Amharic texts
english_sequences, english_tokenizer = preprocess_texts(english_texts)
amharic_sequences, amharic_tokenizer = preprocess_texts(amharic_texts)


Model Development

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding

def build_model(input_vocab_size, output_vocab_size, embedding_dim, units):
    # Encoder
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_vocab_size, embedding_dim)(encoder_inputs)
    encoder_lstm = LSTM(units, return_state=True)
    encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
    encoder_states = [state_h, state_c]

    # Decoder
    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(output_vocab_size, embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = Dense(output_vocab_size, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    # Model
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    return model

input_vocab_size = len(english_tokenizer.word_index) + 1
output_vocab_size = len(amharic_tokenizer.word_index) + 1
embedding_dim = 256
units = 512

model = build_model(input_vocab_size, output_vocab_size, embedding_dim, units)
model.compile(optimizer='adam', loss='categorical_crossentropy')

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 256)            2560      ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 256)            2816      ['input_2[0][0]']             
                                                                                              

 Training the Model

In [7]:
# One-hot encode the target sequences
amharic_sequences_onehot = tf.keras.utils.to_categorical(amharic_sequences[:, 1:], num_classes=output_vocab_size)

# Train the model
history = model.fit(
    [english_sequences, amharic_sequences[:, :-1]],
    amharic_sequences_onehot,
    batch_size=2,
    epochs=500,
    validation_split=0.2
)


Epoch 1/500
2/2 [==============================] - 0s 178ms/step - loss: 1.6570e-05 - val_loss: 1.3197
Epoch 2/500
2/2 [==============================] - 0s 144ms/step - loss: 1.6510e-05 - val_loss: 1.3200
Epoch 3/500
2/2 [==============================] - 0s 170ms/step - loss: 1.6466e-05 - val_loss: 1.3203
Epoch 4/500
2/2 [==============================] - 0s 217ms/step - loss: 1.6406e-05 - val_loss: 1.3207
Epoch 5/500
2/2 [==============================] - 0s 181ms/step - loss: 1.6376e-05 - val_loss: 1.3210
Epoch 6/500
2/2 [==============================] - 0s 181ms/step - loss: 1.6317e-05 - val_loss: 1.3213
Epoch 7/500
2/2 [==============================] - 0s 240ms/step - loss: 1.6257e-05 - val_loss: 1.3216
Epoch 8/500
2/2 [==============================] - 0s 201ms/step - loss: 1.6183e-05 - val_loss: 1.3220
Epoch 9/500
2/2 [==============================] - 0s 184ms/step - loss: 1.6168e-05 - val_loss: 1.3223
Epoch 10/500
2/2 [==============================] - 0s 215ms/step - loss:

5. Evaluation

In [8]:
from nltk.translate.bleu_score import sentence_bleu

def evaluate_model(model, input_text, tokenizer, max_length):
    # Tokenize and pad input text
    sequence = tokenizer.texts_to_sequences([input_text])
    sequence = pad_sequences(sequence, maxlen=max_length, padding='post')

    # Predict translation
    prediction = model.predict([sequence, np.zeros((1, max_length))])
    predicted_sequence = prediction.argmax(axis=-1)

    # Convert sequence back to text
    reverse_tokenizer = {v: k for k, v in tokenizer.word_index.items()}
    predicted_text = ' '.join([reverse_tokenizer[idx] for idx in predicted_sequence[0] if idx != 0])
    return predicted_text

# Example usage
input_text = "Hello world"
max_length = 10
predicted_text = evaluate_model(model, input_text, english_tokenizer, max_length)
print("Predicted translation:", predicted_text)


1/1 [==============================] - 1s 890ms/step
Predicted translation: good good good


 Save and Load the Model


In [9]:
model.save('translation_model.h5')

# Save tokenizers
import pickle
with open('english_tokenizer.pkl', 'wb') as f:
    pickle.dump(english_tokenizer, f)

with open('amharic_tokenizer.pkl', 'wb') as f:
    pickle.dump(amharic_tokenizer, f)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


To load the model and tokenizers

In [10]:
from tensorflow.keras.models import load_model
import pickle

# Load model
model = load_model('translation_model.h5')

# Load tokenizers
with open('english_tokenizer.pkl', 'rb') as f:
    english_tokenizer = pickle.load(f)

with open('amharic_tokenizer.pkl', 'rb') as f:
    amharic_tokenizer = pickle.load(f)
